In [16]:
from configs.config import configs
from langchain import OpenAI 
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.agents import initialize_agent
from langchain.agents import Tool
from langchain.tools import BaseTool

In [17]:
OpenAI.api_key = configs.get("OPENAI_API_KEY')

# Set up the turbo LLM
turbo_llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo',
    verbose=True
)

In [22]:
import os
import json

class BookAppointmentTool(BaseTool):
    name = 'Book Appointment'
    description = """Useful when you need to book/schedule an appointment for a patient with a doctor at your premises.
        INPUT should be a json string in the following format -
        "{
                'patient_name' : '<patient_name>',
                'patient_email' : '<patient_email>',
                'patient_phone' : '<patient_phone_number>',
                'patient_case' : '<patient_case_description>',
                'doctor_name' : '<name_of_doctor_to_visit>',
                'department_name' : '<department_name>',
                'appointment_date' : '<date_of_appointment_in_dd/mm/yy>',
                'appointment_time' : '<time_of_appointment_in_hh:mm am/pm>',
                'send_email' : <whether_to_send_email_or_not>,
                'send_sms' : <whether_to_send_sms_or_not>
        }"
        You must ensure that you get all the necessary information about the patient and appointment mentioned above before using this tool.
        Ask for these details in a conversational manner, that is one information at a time.
        The tool must be initalized only when all 10 parameters contains values. 
        
        Example Input Description - 
        
        "{
                'patient_name' : 'Roy',
                'patient_email' : 'roy@example.com',
                'patient_phone' : '555-555-444',
                'patient_case' : 'Dental-checkup',
                'doctor_name' : 'Dr Mike',
                'department_name' : 'Dental',
                'appointment_date' : '01-10-2023',
                'appointment_time' : '3:00 pm',
                'send_email' : True,
                'send_sms' : False
        }"
        
        """

    def book_patient_appointment(details):
        try:
            if not isinstance(details, dict):
                return "Aborting...Details not parsable."

            # Create the directory if it doesn't exist
            if not os.path.exists('appointments'):
                os.makedirs('appointments')
            
            # Define the file path
            file_path = 'appointments/appointments.json'

            # Load existing data if the file exists, or create an empty list
            if os.path.exists(file_path):
                with open(file_path, 'r') as file:
                    appointments_list = json.load(file)
            else:
                appointments_list = []
            
            # Add the new appointment details to the list
            appointments_list.append(details)
            
            # Write the updated list back to the file
            with open(file_path, 'w') as file:
                json.dump(appointments_list, file, indent=4)
            
            # Return a success message
            return "Appointment booked successfully!"

        except Exception as e:
            return "Unexpected error occured while trying to book. Please try again later."
    

    def _run(self, details: str):
        if details:
            details_dict = json.loads(details)
            response = self.book_patient_appointment(details_dict)
        else:
            response = "Invalid details"
        
        return response

    def _arun(self, details: str):
        raise NotImplementedError("This tool does not support async")

In [21]:
scheduler = BookAppointmentTool()

In [ ]:
fixed_prompt = """

You are Kyle, an outbound phone calling appointment scheduler bot.
Your agenda is to manage the reception at XYZ clinic.
You should collect all the *Necessary Information* of the individual you are talking with which is named as patient and schedule an doctor's appointment for the treatment which the patient wants and the type of doctor which patient wants to visit at "XYZ" Hospital. You should first greet the patient and ask his name.



"""